# ArcGIS Online Replace Layer in a Web Map

Description: The user will provide the layer they wish to replace, and the layer they wish to replace it with. Then the notebook will loop through all the maps searching for the provided layer, and it will replace it with the other provided layer.
    
Created on: 5/21/24

Purpose: This is to streamline the ability to replace layers in webmaps in ArcGIS Online. 

Authored By: Joe Guzi

Previous Production Date: 7/8/24

Production Date: 7/18/24

Note: You can use the ArcGIS Online Web Map Services Audit notebook, below, to get a preview of all of the maps that will be updated by this notebook. Simply run the notebook then filter on the layer you want to replace, and you will have an inventory of all of the maps that will be updated: https://www.arcgis.com/home/item.html?id=72ce7ff61fc5480d850ed68de29f1d9c 

- 7/3/24 - Minor update for proofreading.
- 7/8/24 - Added sign in option "Connect with current AGO Credentials to this organization" to allow the notebook to be uploaded to AGO as a hosted notebook and run from AGO.
- 7/18/24 - rearranged the order of the remove and readd layers to accomodate hosted feature layers with multiple layers. It does not work if the layers are grouped. 
- 8/22/24 - Reworked the code to based on some inspiriation from Glen Bambrick.
    - You can only replace the layers in a webmap using URLs
    - This new method has a more streamlined approach to handling the order of the layers in the WebMap
    - Thank you Glen!
- 9/4/24 - Removed pop-up configuration from ReplacmentLayerFLDict

## Code inspired by:

ArcGIS Online WAB Widget Audit Notebook - https://www.arcgis.com/home/item.html?id=f2609c3d733945c58297c9b96cd1998f

ArcGIS Online Web Map Services Audit Notebook - https://www.arcgis.com/home/item.html?id=72ce7ff61fc5480d850ed68de29f1d9c

ArcGIS Online Basemap Replacement Notebook - https://www.arcgis.com/home/item.html?id=90cc2d7419f947d3ae55b44fd3090b24

https://github.com/Esri/arcgis-python-api/blob/conference_talks/talks/DevSummit2024/Managing_Web_Maps_and_Layers_Using_ArcGIS_API_for_Python/Authoring_and_Editing_Webmap.ipynb  

https://community.esri.com/t5/arcgis-api-for-python-questions/reorder-webmap-layers-with-python/td-p/797542 

https://developers.arcgis.com/python/samples/using-and-updating-GIS-content/ 


# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

print("Define Variables")
LayerURLToReplace = ""
ReplacementLayerURL = ""
print("Define Variables Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise
Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password, verify_cert=False)
print(gis)
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Layer you want to replace
Ok so here's the deal... this is set up to define the Layers by the URL of the Layer. 

In [ ]:
SubmittedLayerURLToReplace = input("What is the URL of the Layer you want to replace? ")

# Step 4: Define the Replacement layer By Item ID
Ok so here's the deal... this is set up to define the Layers by URL only.

In [ ]:
SubmittedReplacementLayerURL = input("What is the Replacement Layer's URL? ")

# Step 5: Define Functions

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

def CheckSubmittedLayers(LayerURLToReplace,ReplacementLayerURL):
    if LayerURLToReplace == "":
        LayertoReplace = ""
        print("Define Layer to replace by URL.")
    elif LayerURLToReplace != "":
        LayertoReplace = LayerURLToReplace
        print("Layer to Replace: " + str(LayertoReplace))

    if ReplacementLayerURL == "":
        ReplacmentLayer = ""
        print("Define Replacement Layer by URL.")
    elif ReplacementLayerURL != "":
        ReplacmentLayer = ReplacementLayerURL
        print("Replacement Layer: " + str(ReplacmentLayer))
        
    return LayertoReplace, ReplacmentLayer

def ReplacementLayerPrep(ReplacmentLayerParam):
    print("Replacement Layer Prep...")
    ReplacmentLayerFL = FeatureLayer(ReplacmentLayerParam, gis=gis)
    ReplacmentLayerFLTitle = ReplacmentLayerFL.properties.name
    #print(ReplacmentLayerFLTitle)
    ReplacmentLayerFLItem_id = ReplacmentLayerFL.properties.serviceItemId 
    #print(ReplacmentLayerFLItem_id)
    ReplacmentLayerFLfield_Infos = []
    for field in ReplacmentLayerFL.properties.fields:
        # create a dictionary for the fieldInfos entry
        field_dict = {
            "fieldName": field["name"],
            "isEditable": field["editable"],
            "label": field["name"],
            "visible": True
        }
        # append into the field_infos list
        ReplacmentLayerFLfield_Infos.append(field_dict)
        ReplacmentLayerFLDict = {
            "id" : None,
            "title" : ReplacmentLayerFLTitle,
            "url" : ReplacmentLayer,
            "itemId" : ReplacmentLayerFLItem_id,
            "layerType" : "ArcGISFeatureLayer"
        }
    #print(ReplacmentLayerFLDict)
    print("Replacement Layer Prep Complete!")
    return ReplacmentLayerFLDict

print("Function Section Complete!")

# Step 6: Audit Web Maps and Replace Layers

In [ ]:
LayertoReplace, ReplacmentLayer = CheckSubmittedLayers(SubmittedLayerURLToReplace, SubmittedReplacementLayerURL)

if LayertoReplace != "" and ReplacmentLayer != "":
    s = ""
    print("Layers defined...")
    # Layer Prep
    ReplacmentLayerFLDict = ReplacementLayerPrep(ReplacmentLayer)
    print("Start Searching...")
    Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000) # All Web Maps
    #Audit = gis.content.search(query="id:InsertItemIdForTesting", item_type="Web Map", max_items = 10000) # Item Id of a Web Map
    for Item in Audit:
        # Defining variables for each Item
          # New Operational Layer list to update Web Map
        new_ol = []
          # List to gather update Statuses
        UpdateStatusList= []
          # Update Status
        UpdateStatus = ""
        # Defining variables for each Item Complete!
        ItemTitle = str(Item["title"].replace(',', ""))
        ItemID = str(Item["id"])
        print(ItemID)
        ItemType = str(Item["type"])
        ItemOwner = str(Item["owner"])
        ItemAccess = str(Item["access"])
        ItemContentStatus = str(Item.get("contentStatus"))
        ItemData = Item.get_data(try_json=True)
        #print(ItemData)
        for ItemInfo in ItemData:
            if ItemInfo == "operationalLayers":
                layers = ItemData["operationalLayers"]
                for layer in layers:
                    #print (layer)
                    s += ItemTitle.replace(",", "") + ","
                    s += ItemID + ","
                    s += ItemType + ","
                    s += ItemOwner + ","
                    s += ItemAccess + ","
                    s += "Operational Layer" + "," 
                    LayerID = str(layer.get("id"))
                    s += LayerID + ","
                    #print(LayerID)
                    s += str(layer.get("title")).replace(",", "") + ","
                    s += str(layer.get("layerType")) + ","
                    s += str(layer.get("visibility")) + ","
                    LayerURL = str(layer.get('url'))
                    s += LayerURL + ","
                    #print(LayerURL)
                    LayerItemID = str(layer.get('itemId'))
                    #print(LayerItemID)
                    # Search Replacement Layer by Item Url
                    if LayerURL != "None":
                        s += LayerURL + ","
                        if LayertoReplace == LayerURL:
                            #print("ready to replace layer")
                            webmap = WebMap(Item)
                            s += ReplacmentLayer + ","
                            ReplacmentLayerFLDict['id'] = LayerID
                            # append the swap layer to the operationalLayers list (new_ol)
                            new_ol.append(ReplacmentLayerFLDict)
                            UpdateStatus = "Updated"
                            UpdateStatusList.append(UpdateStatus)
                            s += UpdateStatus + ","
                            #print("UpdateStatus: " + UpdateStatus)
                        else:
                            new_ol.append(layer)
                            s += ","
                            UpdateStatus = "NotUpdated"
                            UpdateStatusList.append(UpdateStatus)
                            s += UpdateStatus + ","
                            #print("UpdateStatus: " + UpdateStatus)
                    else:
                        s += ","
                    s += "\n"
        #print(UpdateStatusList)
        if "Updated" in UpdateStatusList:
            print("Need to Update")
            # update webmap definition
            webmap.definition.operationalLayers = new_ol
            # set the item_properties dictionary
            item_properties = {"text":webmap.definition}
            # update the wm_item
            print("Update the Web Map Item")
            status = Item.update(item_properties=item_properties)
            print("WebMap Updated = " + status)

                
        else:
            print("Item Skipped... Does not need to be updated.")
else:
    print("Layers are not defined go back to steps 3 and 4.")
print("Audit Section Complete!")

# Step 7: (optional) Write the Audit Results to csv

In [ ]:
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Write data To CSV File")
    output.write("MapTitle,MapId,ItemType,ItemOwner,ItemAccess,MapLocation,LayerID,LayerTitle,LayerType,LayerVisisbility,LayerURL,ItemID,ReplacementLayerItemID,UpdateStatus\n")
    output.writelines(s)
    print("Write data To CSV File Complete!")